In [42]:
import pandas as pd
import plotly.graph_objects as go

In [43]:
tracks = pd.read_csv("D:/Study/Monash/FIT3162/Resonance/src/data/features.csv")
artists = pd.read_csv("D:/Study/Monash/FIT3162/Resonance/src/data/artists.csv")

In [50]:
tracks['loudness'] = (tracks['loudness'] - min(tracks['loudness'])) / (max(tracks['loudness'])-min(tracks['loudness']))
df = tracks.sort_values(by=['loudness'])

In [64]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=df['loudness'], y=df['danceability'], name="danceability", fill='tonexty'),
    secondary_y=False
)

fig.add_trace(
    go.Scatter(x=df['loudness'], y=df['energy'], name="energy", fill='tonexty'),
    secondary_y=True
)

# Add figure title
fig.update_layout(
    title_text="Double Y Axis Example"
)

# Set x-axis title
fig.update_xaxes(title_text="Loudness")

# Set y-axes titles
fig.update_yaxes(title_text="<b>danceability</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>energy</b>", secondary_y=True)

fig.update_layout(
    width=1000,
    height=500,
)

fig.show()